##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Grappler による TensorFlow グラフ最適化

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/graph_optimization"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/graph_optimization.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/graph_optimization.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/graph_optimization.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

## 概要

TensorFlow uses both graph and eager executions to execute computations. A `tf.Graph` contains a set of `tf.Operation` objects (ops) which represent units of computation and `tf.Tensor` objects which represent the units of data that flow between ops.

Grappler is the default graph optimization system in the TensorFlow runtime. Grappler applies optimizations in graph mode (within `tf.function`) to improve the performance of your TensorFlow computations through graph simplifications and other high-level optimizations such as inlining function bodies to enable inter-procedural optimizations. Optimizing the `tf.Graph` also reduces the device peak memory usage and improves hardware utilization by optimizing the mapping of graph nodes to compute resources.

`tf.Graph`の最適化をより細かく制御するには、`tf.config.optimizer.set_experimental_options()`を使用します。


## 使用可能なグラフオプティマイザ

Grappler は`MetaOptimizer`と呼ばれるトップレベルのドライバを通してグラフの最適化を実行します。TensorFlowでは、次のグラフオプティマイザの使用が可能です。

- *定数折り畳みオプティマイザ -* 可能な場合にグラフの定数ノードを折り畳むことによりテンソルの値を静的に推測し、定数を使用して結果をマテリアライズします。
- *算術オプティマイザ -* 一般的な副次式を消去し、算術ステートメントを簡略化することにより、算術演算を簡略化します。
- *レイアウトオプティマイザ -* テンソルのレイアウトを最適化することにより、畳み込みなどのデータフォーマットに依存する演算をより効率的に実行します。
- *リマッパーオプティマイザ -* 一般的に発生するサブグラフを最適化された結合モノリシックカーネルで置き換えることにより、サブグラフをより効率的な実装に再マッピングします。
- *メモリオプティマイザ -* グラフを分析して各演算のピークメモリ使用量を確認し、CPU-GPU メモリコピー演算を挿入してGPUメモリをCPUのメモリにスワップすることにより、ピークメモリ使用量を減らします。
- *依存オプティマイザ -* 制御の依存関係を削除または並べ替えすることにより、モデルステップのクリティカルパスを短縮、または他の最適化を有効化します。 Identity などの実質的な NoOp ノードも削除します。
- *プルーニングオプティマイザ -* グラフからの出力に影響を与えないノードを削除します。通常はこれを最初に実行することにより、グラフのサイズを縮小化し、他の Grappler パスの処理を高速化します。
- *関数オプティマイザ -* TensorFlow プログラムの関数ライブラリを最適化し、関数本体をインライン展開することにより、他のプロシージャ間最適化を可能にします。
- *形状オプティマイザ -* 形状および形状関連の情報に関する演算のサブグラフを最適化します。
- *Autoparallel optimizer -* Automatically parallelizes graphs by splitting along the batch dimension. This optimizer is turned OFF by default.
- *Loop optimizer -* Optimizes the graph control flow by hoisting loop-invariant subgraphs out of loops and by removing redundant stack operations in loops. Also optimizes loops with statically known trip counts and removes statically known dead branches in conditionals.
- *スコープアロケーターオプティマイザ -* スコープアロケーターを導入することにより、データの移動を削減し、一部の演算を統合します。
- *ホスト固定オプティマイザ -* 小さな演算を CPU にスワップします。このオプティマイザはデフォルトではオフです。
- *Auto mixed precision optimizer -* Converts data types to float16 where applicable to improve performance. Currently applies only to GPUs.
- *デバッグストリッパー -* `tf.debugging.Assert`、 `tf.debugging.check_numerics`、`tf.print`などのデバッグ演算関連のノードをストリップします。このオプティマイザはデフォルトではオフです。

## セットアップ


In [ ]:
import numpy as np
import timeit
import traceback
import contextlib


import tensorflow as tf

オプティマイザの状態を簡単に切り替えるためのコンテキストマネージャを作成します。

In [ ]:
@contextlib.contextmanager
def options(options):
  old_opts = tf.config.optimizer.get_experimental_options()
  tf.config.optimizer.set_experimental_options(options)
  try:
    yield
  finally:
    tf.config.optimizer.set_experimental_options(old_opts)

## Grappler 使用の有無による実行パフォーマンスを比較する

TensorFlow 2 以降は、デフォルトで [Eager](../eager.md) Execution が実装されています。デフォルトの実行を Graph モードに切り替えるには、デフォルトで`tf.function`を使用します。Grappler はバックグラウンドで自動的に実行し、上記のグラフオプティマイザを適用して実行パフォーマンスを向上させます。


### 定数折り畳みオプティマイザ

導入的な例として、定数に対して演算を実行し、出力を返す関数を 1 つ考えます。

In [ ]:
def test_function_1():
  @tf.function
  def simple_function(input_arg):
    print('Tracing!')
    a = tf.constant(np.random.randn(2000,2000), dtype = tf.float32)
    c = a
    for n in range(50):
      c = c@a
    return tf.reduce_mean(c+input_arg)

  return simple_function

定数折り畳みオプティマイザをオフにして、関数を実行します。

In [ ]:
with options({'constant_folding': False}):
  print(tf.config.optimizer.get_experimental_options())
  simple_function = test_function_1()
  # Trace once
  x = tf.constant(2.2)
  simple_function(x)
  print("Vanilla execution:", timeit.timeit(lambda: simple_function(x), number = 1), "s")

定数折り畳みオプティマイザを有効にして関数を再度実行し、関数実行の高速化を観察します。

In [ ]:
with options({'constant_folding': True}):
  print(tf.config.optimizer.get_experimental_options())
  simple_function = test_function_1()
  # Trace once
  x = tf.constant(2.2)
  simple_function(x)
  print("Constant folded execution:", timeit.timeit(lambda: simple_function(x), number = 1), "s")

### デバッグ ストリッパー オプティマイザ

入力引数の数値をチェックし、それを返す単純な関数を見てみましょう。 

In [ ]:
def test_function_2():
  @tf.function
  def simple_func(input_arg):
    output = input_arg
    tf.debugging.check_numerics(output, "Bad!")
    return output
  return simple_func

まず最初に、デバッグストリッパーオプティマイザをオフにして関数を実行します。 

In [ ]:
test_func = test_function_2()
p1 = tf.constant(float('inf'))
try:
  test_func(p1)
except tf.errors.InvalidArgumentError as e:
  traceback.print_exc(limit=2)

`test_func`に対する`Inf`引数のため、`tf.debugging.check_numerics`には無効な引数エラーが発生します。

デバッグストリッパーオプティマイザを有効にして、関数を再度実行します。 

In [ ]:
with options({'debug_stripper': True}):
  test_func2 = test_function_2()
  p1 = tf.constant(float('inf'))
  try:
    test_func2(p1)
  except tf.errors.InvalidArgumentError as e:
    traceback.print_exc(limit=2)

デバッグストリッパーオプティマイザがグラフから`tf.debug.check_numerics`ノードを取り除き、エラーを発生させることなく関数を実行します。 

## まとめ

TensorFlow ランタイムは Grappler を使用して、実行前にグラフを自動的に最適化します。`tf.config.optimizer.set_experimental_options`を使用すると、様々なグラフオプティマイザを有効または無効にすることができます。

Grappler に関する詳しい情報は、<a class="external" href="http://web.stanford.edu/class/cs245/slides/TFGraphOptimizationsStanford.pdf">TensorFlow グラフの最適化</a>をご覧ください。